In [188]:
# Auxiliar imports
import sys, os, time, importlib
import matplotlib.pyplot as plt
import numpy as np

# Gym imports
import gym
from gym.vector import SyncVectorEnv

# PyTorch imports
import torch
from torch import nn, optim

# Custom imports
sys.path.append(os.path.abspath('..')) # Add parent directory to path

import ppo_network
importlib.reload(ppo_network) # Prevents caching issues with notebooks
from ppo_network import PPONetworkDiscrete

import ppo
importlib.reload(ppo) # Prevents caching issues with notebooks
from ppo import PPODiscrete

import ppo_restored
importlib.reload(ppo_restored) # Prevents caching issues with notebooks
from ppo_restored import PPOWrapper

import hp_optimizer
importlib.reload(hp_optimizer) # Prevents caching issues with notebooks
from hp_optimizer import HPOptimizer

In [189]:
# LunarLander environment
env_id = 'LunarLander-v2'
max_episode_steps = 1024
num_envs = 16

env_kwargs = {
    'id': env_id,
    'max_episode_steps': max_episode_steps,
}

# Create vectorized environment
envs_vector = SyncVectorEnv([lambda: gym.make(**env_kwargs)] * num_envs)
states, infos = envs_vector.reset()

In [190]:
# Policy-Value Network
# TODO - Move to PPO-kwargs
input_dims = 8
output_dims = 4

shared_hidden_dims = [1024, 512, 256]
shared_norm = nn.LayerNorm
shared_activation = nn.SiLU

policy_hidden_dims = [256, 128, 64]
policy_norm = nn.LayerNorm
policy_activation = nn.SiLU

value_hidden_dims = [256, 128, 64]
value_norm = nn.LayerNorm
value_activation = nn.SiLU

network_kwargs = {
    'input_dims': input_dims,
    'output_dims': output_dims,
    
    'shared_hidden_dims': shared_hidden_dims,
    'shared_norm': shared_norm,
    'shared_activation': shared_activation,
    
    'policy_hidden_dims': policy_hidden_dims,
    'policy_norm': policy_norm,
    'policy_activation': policy_activation,
    
    'value_hidden_dims': value_hidden_dims,
    'value_norm': value_norm,
    'value_activation': value_activation,
}

network = PPONetworkDiscrete(**network_kwargs)

In [191]:
# Test forward passes
now = time.time()
for _ in range(10):
    states_tensor = torch.tensor(states, dtype=torch.float32)
    policy, value = network(states_tensor)
    
    actions_dist = torch.distributions.Categorical(logits=policy)
    actions = actions_dist.sample().numpy()
    
    states, rewards, dones, truncateds, infos = envs_vector.step(actions)
    #print(dones)

print(
    f'Elapsed time: per vectorized env: {(time.time() - now)/num_envs:.2f} s'
    )

Elapsed time: per vectorized env: 0.00 s


/Users/axelsolhall/miniconda3/envs/PPOgym/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [192]:
# PPO 

lr = 3e-4
final_lr = 5e-6

gamma = 0.995
lam = 0.99

clip_eps = 0.25
final_clip_eps = 0.01

value_coef = 0.7

entropy_coef = 0.1
final_entropy_coef = 0.025

batch_size = 256 # TODO - rename to mini_batch
batch_epochs = 8
batch_shuffle = True
seperate_envs_shuffle = True

iterations = 2048 # 2048 # TODO - rename to batch

reward_normalize = True
truncated_reward = -300

debug_prints = False

ppo_kwargs = {
    'num_envs': num_envs,
    'lr': lr,
    'final_lr': final_lr,
    'gamma': gamma,
    'lam': lam,
    'clip_eps': clip_eps,
    'final_clip_eps': final_clip_eps,
    'value_coef': value_coef,
    'entropy_coef': entropy_coef,
    'final_entropy_coef': final_entropy_coef,
    'batch_size': batch_size,
    'batch_epochs': batch_epochs,
    'batch_shuffle': batch_shuffle,
    'seperate_envs_shuffle': seperate_envs_shuffle,
    'iterations': iterations,
    'reward_normalize': reward_normalize,
    'truncated_reward': truncated_reward,
    'debug_prints': debug_prints,   
}

ppo = PPOWrapper(envs_vector, network, **ppo_kwargs)
#ppo = PPODiscrete(envs_vector, network, **ppo_kwargs)

# now = time.time()
ppo.train(generations=100)
# after = time.time()
# print(f'Elapsed time: {after - now:.2f} s')


### 5 generations ###
# not specifed  - 48s
# cpu:          - 236s
# MPS:          - 110s
#? So slow if device is selected, default is sooo much faster

Generation    0 - Reward:  -412.95, w/o trunc.:  -112.95
Generation    1 - Reward:   -64.94, w/o trunc.:    85.06
Generation    2 - Reward:  -239.81, w/o trunc.:  -127.31
Generation    3 - Reward:   -95.70, w/o trunc.:    -1.95
Generation    4 - Reward:  -185.87, w/o trunc.:   114.13
Generation    5 - Reward:   -16.50, w/o trunc.:   133.50
Generation    6 - Reward:     2.10, w/o trunc.:   152.10


KeyboardInterrupt: 

In [142]:
# Hyperparameter optimization
hp_optimizer = HPOptimizer(
    env_kwargs=env_kwargs,
    num_envs=num_envs,
    network_class = PPONetworkDiscrete,
    network_kwargs=network_kwargs,
    ppo_class=PPODiscrete,
    ppo_kwargs=ppo_kwargs,
)

parameters = [
    #'batch_shuffle',
    'reward_normalize',
    'seperate_envs_shuffle',
    #('gamma', [0.99, 0.995, 0.999]), 
    ('lam', [0.95, 0.975, 0.99]),
    #('final_clip_eps', [0.002, 0.01, 0.05]),
    ('final_lr', [1e-5, 1e-6, 1e-7]),
    ('batch_size', [64, 128, 256, 512]),
    ('batch_epochs', [2, 4, 8, 16]),
    ]

# evolutions = hp_optimizer.optimize_hyperparameters(
#     parameters, generations=40, num_trials = 16,
#     )


# hp_optimizer.evolution_video(
#     generations=100, video_folder = 'videos', increments=10, max_frames=max_episode_steps,
#     )